非线性规划——选址问题

线性规划——料场位置固定

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import linprog

#工地横坐标
a = [1.25, 8.75, 0.5, 5.75, 3, 7.25]
#纵坐标
b = [1.25, 0.75, 4.75, 5, 6.5, 7.25]
#水泥日需求列表
d = [3, 5, 4, 7, 6, 11]

#临时料场固定
#临时料场的坐标列表
x1_coord, y1_coord = 5, 1
x2_coord, y2_coord = 2, 7
#记算每个工地到料场1的距离
distance_to_x1 = []
for i in range(len(a)):
    distance = np.sqrt((x1_coord-a[i])**2 + (y1_coord-b[i])**2)
    distance_to_x1.append(distance)
#记算每个工地到料场2的距离
distance_to_x2 = []
for i in range(len(a)):
    distance = np.sqrt((x2_coord-a[i])**2 + (y2_coord-b[i])**2)
    distance_to_x1.append(distance)
#将两个列表合并
f = distance_to_x1 + distance_to_x2

#构造不等式约束的系数矩阵
A = []
#构造对应第一个料场供应各工地水泥量系数的行
row_1 = [1]*6 + [0]*6
A.append(row_1)
#构造对应第二个料场供应各工地水泥量系数的行
row_2 = [0]*6 + [1]*6
A.append(row_2)
#不等式约束右侧常量
b_constraints = [20,20]

#构造等式约束的系数矩阵
eye_matrix_1 = np.eye(6)
eye_matrix_2 = np.eye(6)
A_eq = np.hstack([eye_matrix_1,eye_matrix_2])
#等式约束右侧系数
b_eq = np.array(d)

#变量边界
bound = [(0,None)] * 12

resurt = linprog(f,A_ub=A,b_ub=b_constraints,A_eq=A_eq,b_eq=b_eq,bounds=bound)
x1, fval = resurt.x, resurt.fun
print('第一问解：',x1)
print('第一问目标值：',fval)

第一问解： [ 3.  5.  0.  7.  0.  1.  0.  0.  4.  0.  6. 10.]
第一问目标值： 135.28154179067644


非线性规划——料场位置不定

In [2]:
#目标函数
def objective(x):
    f1 = sum(np.sqrt((x[12]-a)**2 + (x[13]-b)**2)*x[:6])
    f2 = sum(np.sqrt((x[14]-a)**2 + (x[15]-b)**2)*x[6:12])
    return f1+f2
#定义不等式约束条件对应的记算函数
def inequality_constraints_direct(x):
    return np.array([20-sum(x[:6]),20-sum(x[6:12])])
#定义等式约束条件对应的记算函数
def equality_constraints_direct(x):
    return np.array([x[0]+x[6]-3, x[1]+x[7]-5, x[2]+x[8]-4,
                     x[3]+x[9]-7, x[4]+x[10]-6, x[5]+x[11]-11])
#构建约束条件的字典列表，包含不等式和等式约束
cons = [{'type': 'ineq', 'fun': inequality_constraints_direct},  #不等式约束
        {'type': 'eq', 'fun': equality_constraints_direct}]      #等式约束
#变量边界定义
bound2 = [(0,None)]*12 + [(-np.inf,np.inf)]*4

x0_lp = np.hstack([x1, [5,1,2,7]])

res = minimize(fun=objective,x0=x0_lp,constraints=cons,bounds=bound2)
x2_lp,fval2_lp = res.x, res.fun
print('用线性规划结果作为初始值：')
print('解为：',x2_lp)
print('目标值为：',fval2_lp)

用线性规划结果作为初始值：
解为： [3.00000000e+00 5.00000000e+00 4.00000000e+00 7.00000000e+00
 1.00000000e+00 3.20739357e-14 1.08676995e-15 1.83903274e-14
 1.02582644e-13 1.49761239e-15 5.00000000e+00 1.10000000e+01
 5.69588495e+00 4.92847435e+00 7.25000006e+00 7.25000003e+00]
目标值为： 89.31176250158214
